In [1]:
#Import Libraries
import pandas as pd
from pathlib import Path
import pickle

In [3]:
#Import CSV with all data
data_path = Path("historical_data.csv")
historical_data = pd.read_csv(data_path)
historical_data.rename( columns = {"Unnamed: 0":"date"}, inplace = True)
historical_data.head()

,Date,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,SPY,10 YR
0,2008-01-02,32.13,28.34,79.75,28.41,34.94,38.37,41.33,26.10,41.98,144.55,3.91
1,2008-01-03,31.78,28.22,80.21,28.20,35.18,38.58,41.80,26.08,42.08,144.46,3.91
2,2008-01-04,30.78,28.04,78.01,27.39,34.83,37.60,40.74,25.07,42.30,140.93,3.88
3,2008-01-07,30.93,28.55,77.04,27.55,35.57,37.34,40.25,24.97,43.25,141.46,3.86
4,2008-01-08,30.32,28.38,76.08,26.54,35.81,36.39,39.60,24.21,43.33,138.81,3.86


In [4]:
# Define a function that returns the indexes where the 10 yr treasury momentum crossed a certain threshold
momentum_window=10
reaction_window=90
spike_percent= 0.187


def spike_selector(df):
    signal1 = 0
    daycount1 = 0
    indexes = []
    df["mmt10"] = (df["10 YR"] - df["10 YR"].shift(momentum_window))
    df["roll_10"] = df["mmt10"].rolling(20).mean()
    no_idx = df.reset_index()
    for index, row in zip(no_idx.index, no_idx["roll_10"]):
        if signal1 == 1:
            daycount1 += 1
            if daycount1 > reaction_window:
                signal1 = 0
                daycount1 = 0
        if row >= spike_percent and signal1 == 0:
            indexes.append(index)
            signal1 = 1
    df.drop(columns = ["mmt10", "roll_10"], inplace = True)
    return indexes



In [5]:
#Create empty dictionary to store new dataframes.  Then slice 90 days following each treaury yeild increase selected by the above function
yield_spikes_dict = {}
for index in spike_selector(historical_data):
    start = index
    end = start + 90
    yield_spikes_dict[index] = historical_data.iloc[start: end]

print(yield_spikes_dict.keys())

dict_keys([269, 360, 498, 736, 1357, 2221, 3292])


In [6]:
print(spike_selector(historical_data))


[269, 360, 498, 736, 1357, 2221, 3292]


In [7]:
#For loop to update keys
count = 1
final_dict={}
spike_list=[]
for key, value in yield_spikes_dict.items():
    new_df = yield_spikes_dict[key]
    new_df.reset_index(inplace=True)
    new_df.drop(columns=['Date','index'], axis=1, inplace=True)
    if new_df["10 YR"].count()>=85:
        spike_date=str(historical_data.iloc[key].Date)
        final_dict[spike_date]=new_df
        count +=1
    
print(final_dict.keys())

dict_keys(['2009-01-29', '2009-06-10', '2009-12-29', '2010-12-10', '2013-06-07', '2016-11-21'])


C:\Users\George Koulogeorgas\.conda\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
#Concat dfs for all spikes into one df, index by keys and export to pickle object
spikes_df= pd.concat(final_dict.values(), axis = 1, keys = final_dict.keys())
spikes_df.to_pickle('spikes_df.pickle')
spikes_df.head()

2009-01-29                                                                 \
         XLY    XLP    XLE   XLF    XLV     XLI    XLB    XLK    XLU    SPY   
0      19.81  22.83  47.54  9.49  26.53  16.952  21.92  15.12  29.61  84.57   
1      19.28  22.11  46.89  9.28  26.20  16.590  21.06  14.74  28.93  82.69   
2      19.11  21.96  46.15  9.25  26.45  16.211  20.77  14.96  29.07  82.59   
3      19.66  22.43  46.97  9.03  27.08  16.509  21.15  15.22  29.30  83.81   
4      19.21  21.83  47.29  8.95  26.91  16.421  21.50  15.21  29.53  83.31   

   ... 2016-11-21                                                              \
   ...        XLP      XLE    XLF    XLV    XLI    XLB    XLK     XLU     SPY   
0  ...     50.850  72.8100  22.24  69.91  59.61  48.89  47.85  46.780  220.11   
1  ...     51.100  72.7650  22.25  68.95  59.88  49.19  47.98  46.940  220.53   
2  ...     50.810  73.1200  22.38  69.20  60.32  49.43  47.79  46.495  220.73   
3  ...     51.205  72.7300  22.43  69.46  60.65  49.58  47.99  47.155  221.54   
4  ...     51.320  71.6901  22.13  68.92  60.23  49.37  48.03  48.085  220.54   

         
  10 YR  
0  2.33  
1  2.31  
2  2.36  
3  2.36  
4  2.32  

[5 rows x 66 columns]

In [9]:
# Calculate average percent change and groupby sector 
spikes_df.head()

spikes_pct_change= spikes_df.pct_change()
spikes_pct_change.to_pickle('spikes_pct_change.pickle')
    
grouped_df=spikes_pct_change.groupby(level = 1, axis = 1).mean()
grouped_df.head()
grouped_df.to_pickle('grouped_df.pickle')


spikes_pct_change.head()

2009-01-29                                                              \
         XLY       XLP       XLE       XLF       XLV       XLI       XLB   
0        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1  -0.026754 -0.031537 -0.013673 -0.022129 -0.012439 -0.021354 -0.039234   
2  -0.008817 -0.006784 -0.015782 -0.003233  0.009542 -0.022845 -0.013770   
3   0.028781  0.021403  0.017768 -0.023784  0.023819  0.018383  0.018296   
4  -0.022889 -0.026750  0.006813 -0.008859 -0.006278 -0.005330  0.016548   

                                 ... 2016-11-21                                \
        XLK       XLU       SPY  ...        XLP       XLE       XLF       XLV   
0       NaN       NaN       NaN  ...        NaN       NaN       NaN       NaN   
1 -0.025132 -0.022965 -0.022230  ...   0.004916 -0.000618  0.000450 -0.013732   
2  0.014925  0.004839 -0.001209  ...  -0.005675  0.004879  0.005843  0.003626   
3  0.017380  0.007912  0.014772  ...   0.007774 -0.005334  0.002234  0.003757   
4 -0.000657  0.007850 -0.005966  ...   0.002246 -0.014298 -0.013375 -0.007774   

                                                               
        XLI       XLB       XLK       XLU       SPY     10 YR  
0       NaN       NaN       NaN       NaN       NaN       NaN  
1  0.004529  0.006136  0.002717  0.003420  0.001908 -0.008584  
2  0.007348  0.004879 -0.003960 -0.009480  0.000907  0.021645  
3  0.005471  0.003035  0.004185  0.014195  0.003670  0.000000  
4 -0.006925 -0.004236  0.000834  0.019722 -0.004514 -0.016949  

[5 rows x 66 columns]

In [10]:
#Calculate cumulative returns as percentage
returns_df = ((1+grouped_df).cumprod()-1) *100
returns_df.to_pickle('returns_df.pickle')


In [11]:
#Create df which groups the data by sector
sector_list = list(spikes_pct_change.columns.levels[1]) 
sector_dict={}
for sector in sector_list:
    sector_dict[sector]=spikes_pct_change.xs(sector, level=1, axis=1)
sector_df= pd.concat(sector_dict.values(), axis = 1, keys = sector_dict.keys())
sector_df.to_pickle('sector_df.pickle')
sector_df.head()

XLY                                                         \
  2009-01-29 2009-06-10 2009-12-29 2010-12-10 2013-06-07 2016-11-21   
0        NaN        NaN        NaN        NaN        NaN        NaN   
1  -0.026754  -0.009136  -0.003972  -0.005057  -0.002816   0.012243   
2  -0.008817   0.001257  -0.009305  -0.000535  -0.009531   0.000968   
3   0.028781  -0.021766   0.006374  -0.003212  -0.011048   0.002537   
4  -0.022889  -0.022251   0.004000   0.007519   0.018018  -0.007714   

         XLP                                   ...        SPY             \
  2009-01-29 2009-06-10 2009-12-29 2010-12-10  ... 2009-12-29 2010-12-10   
0        NaN        NaN        NaN        NaN  ...        NaN        NaN   
1  -0.031537   0.000857  -0.001120   0.000000  ...  -0.000266   0.000321   
2  -0.006784   0.005563  -0.011589   0.005493  ...  -0.009862   0.000803   
3   0.021403  -0.021277   0.010590   0.001366  ...   0.016870  -0.004412   
4  -0.026750  -0.011304  -0.001123   0.007842  ...   0.002736   0.005560   

                             10 YR                                   \
  2013-06-07 2016-11-21 2009-01-29 2009-06-10 2009-12-29 2010-12-10   
0        NaN        NaN        NaN        NaN        NaN        NaN   
1  -0.000303   0.001908   0.000000  -0.025126  -0.005236  -0.009036   
2  -0.009954   0.000907  -0.038328  -0.018041   0.013158   0.060790   
3  -0.008276   0.003670   0.047101  -0.013123   0.000000   0.011461   
4   0.015083  -0.004514   0.020761  -0.023936  -0.020779  -0.016997   

                         
  2013-06-07 2016-11-21  
0        NaN        NaN  
1   0.023041  -0.008584  
2  -0.009009   0.021645  
3   0.022727   0.000000  
4  -0.026667  -0.016949  

[5 rows x 66 columns]

In [23]:
#equalize sectors to the SP500 price at that time
spike_list = list(sector_df.columns.levels[1])
spy_equalized = {}
for sector in sector_list:
    spy_equalized[sector]={}
    for spike in spike_list:
        spy_equalized[sector][spike] = sector_df[sector][spike] - sector_df['SPY'][spike]
    
new_spy_dict = {}
for sector in sector_list:
    new_spy_dict[sector] = pd.concat(spy_equalized[sector].values(), axis=1, keys = spy_equalized[sector].keys())
    
equalized_df = pd.concat(new_spy_dict.values(), axis=1, keys=new_spy_dict.keys())
equalized_df.head()


XLY                                                         \
  2009-01-29 2009-06-10 2009-12-29 2010-12-10 2013-06-07 2016-11-21   
0        NaN        NaN        NaN        NaN        NaN        NaN   
1  -0.004524  -0.014700  -0.003706  -0.005378  -0.002513   0.010335   
2  -0.007608  -0.001167   0.000557  -0.001338   0.000423   0.000061   
3   0.014009   0.001890  -0.010496   0.001200  -0.002772  -0.001132   
4  -0.016923  -0.009328   0.001264   0.001959   0.002935  -0.003200   

         XLP                                   ...        SPY             \
  2009-01-29 2009-06-10 2009-12-29 2010-12-10  ... 2009-12-29 2010-12-10   
0        NaN        NaN        NaN        NaN  ...        NaN        NaN   
1  -0.009307  -0.004708  -0.000854  -0.000321  ...        0.0        0.0   
2  -0.005575   0.003139  -0.001727   0.004690  ...        0.0        0.0   
3   0.006631   0.002380  -0.006280   0.005778  ...        0.0        0.0   
4  -0.020784   0.001618  -0.003858   0.002282  ...        0.0        0.0   

                             10 YR                                   \
  2013-06-07 2016-11-21 2009-01-29 2009-06-10 2009-12-29 2010-12-10   
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        0.0        0.0   0.022230  -0.030690  -0.004969  -0.009357   
2        0.0        0.0  -0.037118  -0.020465   0.023020   0.059987   
3        0.0        0.0   0.032330   0.010533  -0.016870   0.015874   
4        0.0        0.0   0.026727  -0.011013  -0.023515  -0.022557   

                         
  2013-06-07 2016-11-21  
0        NaN        NaN  
1   0.023345  -0.010492  
2   0.000945   0.020738  
3   0.031003  -0.003670  
4  -0.041750  -0.012435  

[5 rows x 66 columns]

In [24]:
#to pickle
equalized_df.to_pickle('sp_equalized.pickle')

In [25]:
#drop SPY and 10 year
equalized_average = equalized_df.groupby(level=0, axis=1).mean()
equalized_average.drop(columns = ['10 YR'], inplace=True)
equalized_average.head()

,SPY,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.001434,0.003846,0.000603,-0.001820,0.000346,-0.002062,0.003125,0.000049,-0.003414
2,0.0,-0.005320,-0.004939,-0.000540,-0.002230,0.001316,0.000158,0.001195,0.004710,-0.001512
3,0.0,0.001145,0.000942,-0.007723,0.000391,0.002077,0.002701,-0.002443,0.000399,0.000450
4,0.0,0.003197,0.000914,-0.000311,-0.000282,0.000193,-0.003111,0.006313,0.000745,-0.003882


In [26]:
equalized_avg_cum = ((1+equalized_average).cumprod()-1) *100
equalized_avg_cum.to_pickle("sp_equalized_avg.pickle")